Trong phần này sẽ tìm hiểu về cách tính đạo hàm bằng cách sử dụng autograd package trong Pytorch. 

# 1- The Autograd package
Autograd package cung cấp một sự khác biệt tự động cho tất cả các hoạt động của tensor. Rất dễ dàng để sử dụng đạo hàm trong pytorch bằng cách chỉ cho nó biết rằng tensor cần được đạo hàm bằng *requires_grad*. Với việc thiết lập thuộc tính này, các phép toán trên tensor đều được theo dõi trên một đồ thị tính toán.

In [1]:
import torch

x = torch.randn(3, requires_grad=True)
y = x + 2

In [2]:
"""
  - ở cell trên y đã được tạo ra bởi kết quả của phép tính x + 2, vì vậy nó sẽ tạo một thuộc tính grad_fn.
  - grad_fn: tham chiếu đến một hàm đã tạo tensor
"""
print(x) # Đã được tạo ở cell tren -> grad_fn lúc này là None
print(y)
print(y.grad_fn)

tensor([ 0.0990, -0.6665, -0.1077], requires_grad=True)
tensor([2.0990, 1.3335, 1.8923], grad_fn=<AddBackward0>)


In [3]:
# Thực hiện các phép tính khác trên y
z = y * 3
print(z)
z =  z + 3
print(z)

tensor([6.2969, 4.0006, 5.6768], grad_fn=<MulBackward0>)
tensor([9.2969, 7.0006, 8.6768], grad_fn=<AddBackward0>)


# 2 - Tính đạo hàm với lan truyền ngược

Khi hoàn tất quá trình tính toán, ta có thể gọi *.backward()* và tất cả giá trị đạo hàm sẽ được tính toán một cách tự động. Giá trị đạo hàm của những tensor này sẽ được tích lũy vào trong thuộc tính *.grad*. Nó chính là đạo hàm riêng của tensor.

In [4]:
z = z.mean()
z.backward()
print(x.grad) # dz/dx

tensor([1., 1., 1.])


In [5]:
x = torch.randn(5, requires_grad=True)
print(x)
y = x*2
for _ in range(10):
  y = y * 2

print(y)
print(y.shape)

tensor([ 0.4846,  0.0880, -0.5263,  0.2252,  2.1583], requires_grad=True)
tensor([  992.5316,   180.1993, -1077.7963,   461.1786,  4420.2441],
       grad_fn=<MulBackward0>)
torch.Size([5])


In [15]:
w = torch.tensor([0.1, 1.0, 0.0001, 0.01, 0.001], dtype=torch.float32)
y.backward(w)
print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01, 2.0480e+01, 2.0480e+00])


# 3 - Stop a tensor from tracking history

Trong quá trình huấn luyện, khi chúng ta muốn cập nhật trọng số thì thao tác cập nhật này không nên là một phần của phép tính đạo hàm. Chúng ta có 3 sự lựa chọn cho việc dừng quá trình đạo hàm và cập nhật tham số như sau:
- x.requires_grad_false()
- x.detach()
- wrap in with torch.no_grad():

**.requires_grad_(...) thay đổi yêu cầu ngay tại vị trí cần yêu cầu đạo hàm**

In [6]:
# requires_grad_()
a = torch.rand(2, 2)
print(a.requires_grad) # Kiểm tra a đã được yêu cầu tính đạo hàm hay chưa
b = (a*5) / (a-1)
print(b.grad_fn) # Do a chưa tính đạo hàm nên grad_fn lúc này sẽ là None 
a.requires_grad_(True) # thiết lập tính đạo hàm cho a 
print(a.requires_grad) 
b = (a**2).sum()
print(b.grad_fn) # Sau khi thiết lập đạo hàm cho a. Thì phép tính b trên a sẽ nhận được grad_fn

False
None
True


**.detach(): Lấy một tensor mới với nội dung tương tự nhưng không yêu cầu tính đạo hàm**

In [18]:
a = torch.rand(2, 2, requires_grad=True)
print(a.requires_grad)
print(a)
b = a.detach()
print(b.requires_grad)
print(b)

True
tensor([[0.2102, 0.7872],
        [0.6593, 0.0418]], requires_grad=True)
False
tensor([[0.2102, 0.7872],
        [0.6593, 0.0418]])


**wrap in with torch.no_grad()**

In [19]:
a = torch.rand(2, 2, requires_grad=True)
print(a.requires_grad)
with torch.no_grad():
  print((x + 2).requires_grad)

True
False


# 4 - Empty gradients

Với backward() ta sẽ có đạo hàm tích lũy bên trong thuộc tính *.grad*. Chúng ta cần cẩn thận với nó trong quá trình tối ưu.
-> Sử dụng *.zero_()* cho đạo hàm trước khi bắt đầu bước tối ưu - điều này sẽ tránh lưu lại kết quả của lần đạo hàm trước đó. 

In [7]:
weights = torch.ones(4, requires_grad=True)

for epoch in range(5):
  model_output = (weights*5).sum()
  model_output.backward()

  print(weights.grad) 
  # Tối ưu model bằng cách cập nhật trọng số sau khi đạo hàm
  with torch.no_grad():
    weights -= 0.001 * weights.grad
  
  weights.grad.zero_() # sử dụng empty gradients trước khi bắt đầu một lần tối ưu tiếp theo.

tensor([5., 5., 5., 5.])
tensor([5., 5., 5., 5.])
tensor([5., 5., 5., 5.])
tensor([5., 5., 5., 5.])
tensor([5., 5., 5., 5.])
